<a href="https://colab.research.google.com/github/FatemehNMT/Visual-SLAM-Book-Google-Colab/blob/main/ch6_ch9_Ceres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Ceres** is a widely used optimization library for least-square problems. In
Ceres, as users, we only need to define the optimization problem to be solved according
to specific steps and then hand it over to the solver for calculation.\
 \
In order to tell Ceres the definition of the problem, we need to do the following
things:


*   **Defines each parameter block**. The parameter block is usually a trivial vector,
but it can also be defined as a particular structure such as quaternion and Lie
algebra in SLAM. If it is a vector, we need to allocate a double array for each
parameter block to store the variable's value.
*   Then, define the **calculation method** of the residual block. The residual block is usually associated with several parameter blocks, performs some custom
calculations on them, and then returns the residual value. After that, Ceres
will sum the squares residuals, which is used as the overall objective function.
*   In the **residual blocks**, we also need to define the Jacobian calculation method. In Ceres, we can use the “automatic derivative” function or manually specify the Jacobian calculation process. If you want to use automatic derivation, then the residual block must be written in a specific way: the residual calculation should be implemented as a bracketed operator with a template. We will illustrate this point through an example.
*   Finally, add all the **parameter blocks** and **residual blocks** to Ceres's Problem object and call the Solve function to solve it. Before solving, we can pass some configuration information, such as the number of iterations, termination conditions, etc., or use the default configuration.

# **Chapter 5: Nonlinear Optimization**

**Chapter Reference:** https://github.com/gaoxiang12/slambook2/tree/master/ch6

## **Mount**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


## **Install Eigen**

In [ ]:
!git clone https://gitlab.com/libeigen/eigen.git

Cloning into 'eigen'...
remote: Enumerating objects: 128320, done.
remote: Counting objects: 100% (1635/1635), done.
remote: Compressing objects: 100% (587/587), done.
remote: Total 128320 (delta 1055), reused 1583 (delta 1045), pack-reused 126685 (from 1)
Receiving objects: 100% (128320/128320), 107.21 MiB | 30.60 MiB/s, done.
Resolving deltas: 100% (106159/106159), done.


In [ ]:
%cd eigen

/content/eigen


In [ ]:
!mkdir build
%cd build

/content/eigen/build


In [ ]:
!cmake ..

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The Fortran compiler identification is GNU 11.4.0
-- Performing Test standard_math_library_linked_to_automatically
-- Performing Test standard_math_library_linked_to_automatically - Success
-- Standard libraries to link to explicitly: none
-- Performing Test COMPILER_SUPPORT_WERROR
-- Performing Test COMPILER_SUPPORT_WERROR - Success
-- Performing Test COMPILER_SUPPORT_pedantic
-- Performing Test COMPILER_SUPPORT_pedantic - Success
-- Performing Test COMPILER_SUPPORT_Wall
-- Performing T

In [ ]:
!sudo make install

[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/double.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/complex_single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/complex_double.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/xerbla.cpp.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/lsame.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/dspmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/ssbmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_s

## **Install Ceres**

**TEST and Download ceres-solver-2.1.0.tar:**

https://stackoverflow.com/questions/72368717/how-to-download-ceres-solver-2-1-0-tar-gz-file-to-install-ceres-solver-in-ubun

There are **three** files of

SnavelyReprojectionError.h,

common.h,

common.cpp

that must be run before running the main part of **bundle_adjustment_ceres.cpp**

https://github.com/ceres-solver/ceres-solver

http://ceres-solver.org/installation.html

In [ ]:
%cd /content

/content


In [ ]:
!mkdir Ceres
%cd Ceres

/content/Ceres


In [ ]:
# google-glog + gflags
!sudo apt-get install libgoogle-glog-dev libgflags-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgflags2.2 libgoogle-glog0v5 libunwind-dev
The following NEW packages will be installed:
  libgflags-dev libgflags2.2 libgoogle-glog-dev libgoogle-glog0v5
  libunwind-dev
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,207 kB of archives.
After this operation, 7,675 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgflags2.2 amd64 2.2.2-2 [78.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgflags-dev amd64 2.2.2-2 [93.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgoogle-glog0v5 amd64 0.5.0+really0.4.0-2 [60.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunwind-dev amd64 1.3.2-2build2.1 [1,883 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgoogle-glog-dev amd64 0.

In [ ]:
# Use ATLAS for BLAS & LAPACK
!sudo apt-get install libatlas-base-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# Eigen3
!sudo apt-get install libeigen3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libeigen3-doc libmpfrc++-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,056 kB of archives.
After this operation, 9,081 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 1,056 kB in 1s (873 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pac

In [ ]:
# SuiteSparse (optional)
!sudo apt-get install libsuitesparse-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2
  librbio2 libsliplu1 libspqr2 libsuitesparseconfig5 libumfpack5
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2
  librbio2 libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5
  libumfpack5
0 upgraded, 19 newly installed, 0 to remove and 35 not upgraded.
Need to get 22.4 MB of archives.
After this operation, 169 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5

In [ ]:
!sudo apt-get install liblapack-dev libsuitesparse-dev libcxsparse3 libgflags-dev libgoogle-glog-dev libgtest-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
libcxsparse3 is already the newest version (1:5.10.1+dfsg-4build1).
libcxsparse3 set to manually installed.
libgflags-dev is already the newest version (2.2.2-2).
libgoogle-glog-dev is already the newest version (0.5.0+really0.4.0-2).
libsuitesparse-dev is already the newest version (1:5.10.1+dfsg-4build1).
The following additional packages will be installed:
  googletest
The following NEW packages will be installed:
  googletest libgtest-dev
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 792 kB of archives.
After this operation, 5,156 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 googletest all 1.11.0-3 [541 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgtest-dev amd64 1.11.0-3 [250 kB]
Fetched 792 kB in 1s (829 kB/s)
debco

In [ ]:
!git clone https://github.com/ceres-solver/ceres-solver.git

Cloning into 'ceres-solver'...
remote: Enumerating objects: 23130, done.
remote: Counting objects: 100% (543/543), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 23130 (delta 320), reused 202 (delta 199), pack-reused 22587 (from 3)
Receiving objects: 100% (23130/23130), 18.06 MiB | 25.79 MiB/s, done.
Resolving deltas: 100% (16086/16086), done.


In [ ]:
!sudo apt-get install libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libceres2
The following NEW packages will be installed:
  libceres-dev libceres2
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 2,011 kB of archives.
After this operation, 14.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libceres2 amd64 2.0.0+dfsg1-5 [834 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libceres-dev amd64 2.0.0+dfsg1-5 [1,177 kB]
Fetched 2,011 kB in 2s (866 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
deb

In [ ]:
# !unzip '/content/drive/MyDrive/ceres-solver-2.1.0.tar.gz' -d '/ceres-solver/Test_ceres-solver'

!tar -xzf "/content/gdrive/MyDrive/slambook_data/ch10/ceres-solver-2.1.0.tar.gz" -C "/content/Ceres/ceres-solver"


In [ ]:
%cd ceres-solver


# !mkdir build
# !cd build
# !checkout master
# !cmake ..
# # !cmake ../ceres-solver-2.1.0/
# !make -j32
# !sudo make install
# ORRRRR ...

/content/Ceres/ceres-solver


In [ ]:
!mkdir ceres-bin
%cd ceres-bin

/content/Ceres/ceres-solver/ceres-bin


In [ ]:
!cmake ../ceres-solver-2.1.0/

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_BIGOBJ
-- Performing Test HAVE_BIGOBJ - Failed
-- Looking for pow in m
-- Looking for pow in m - found
-- Detected Ceres version: 2.1.0 from /content/Ceres/ceres-solver/ceres-solver-2.1.0/include/ceres/version.h
-- Detected available Ceres threading models: [CXX_THREADS, OPENMP, NO_THREADS]
-- Found Eigen version 3.4.90: /usr/local/share/eigen3/cmake
-- Enabling use of Eigen as a sparse linear algebra library.
CMake Warning (dev) at CMakeLists.txt:230 (find_package):

In [ ]:
!make -j8

[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_2_2.cc.o
[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_2_4.cc.o
[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_2_3.cc.o
[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_2_d.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_3.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_4.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_6.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_9.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_inter

In [ ]:
!make test

Running tests...
Test project /content/Ceres/ceres-solver/ceres-bin
        Start   1: cuda_memcheck_dense_qr_test
Could not find executable cuda-memcheck
Looked in the following places:
cuda-memcheck
cuda-memcheck
Release/cuda-memcheck
Release/cuda-memcheck
Debug/cuda-memcheck
Debug/cuda-memcheck
MinSizeRel/cuda-memcheck
MinSizeRel/cuda-memcheck
RelWithDebInfo/cuda-memcheck
RelWithDebInfo/cuda-memcheck
Deployment/cuda-memcheck
Deployment/cuda-memcheck
Development/cuda-memcheck
Development/cuda-memcheck
Unable to find executable: cuda-memcheck
  1/183 Test   #1: cuda_memcheck_dense_qr_test ...................................***Not Run   0.00 sec
        Start   2: cuda_memcheck_dense_cholesky_test
Could not find executable cuda-memcheck
Looked in the following places:
cuda-memcheck
cuda-memcheck
Release/cuda-memcheck
Release/cuda-memcheck
Debug/cuda-memcheck
Debug/cuda-memcheck
MinSizeRel/cuda-memcheck
MinSizeRel/cuda-memcheck
RelWithDebInfo/cuda-memcheck
RelWithDebInfo/cuda-memcheck
D

In [ ]:
# Optionally install Ceres, it can also be exported using CMake which
# allows Ceres to be used without requiring installation, see the documentation
# for the EXPORT_BUILD_DIR option for more information.
!make install

[ 25%] Built target ceres_internal
[ 27%] Built target ceres
[ 28%] Built target gtest
[ 28%] Built target test_util
[ 28%] Built target array_utils_test
[ 28%] Built target array_selector_test
[ 28%] Built target autodiff_test
[ 28%] Built target autodiff_first_order_function_test
[ 29%] Built target autodiff_cost_function_test
[ 29%] Built target autodiff_local_parameterization_test
[ 30%] Built target autodiff_manifold_test
[ 30%] Built target block_jacobi_preconditioner_test
[ 31%] Built target block_random_access_dense_matrix_test
[ 31%] Built target block_random_access_diagonal_matrix_test
[ 31%] Built target block_random_access_sparse_matrix_test
[ 32%] Built target block_sparse_matrix_test
[ 33%] Built target c_api_test
[ 33%] Built target canonical_views_clustering_test
[ 33%] Built target compressed_col_sparse_matrix_utils_test
[ 34%] Built target compressed_row_sparse_matrix_test
[ 34%] Built target concurrent_queue_test
[ 34%] Built target conditioned_cost_function_test
[ 3

## **Examples**

### **SLAM Book 2: Curve Fitting with Google Ceres**

**Page 114**\
https://github.com/gaoxiang12/slambook2/blob/master/ch6/ceresCurveFitting.cpp

In [ ]:
!pwd

/content/Ceres/ceres-solver/ceres-bin


In [ ]:
!bin/simple_bundle_adjuster ../ceres-solver-2.1.0/data/problem-16-22106-pre.txt

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  4.185660e+06    0.00e+00    1.09e+08   0.00e+00   0.00e+00  1.00e+04        0    9.93e-02    3.28e-01
   1  1.062590e+05    4.08e+06    8.99e+06   5.36e+02   9.82e-01  3.00e+04        1    1.50e-01    4.78e-01
   2  4.992817e+04    5.63e+04    8.32e+06   3.19e+02   6.52e-01  3.09e+04        1    1.19e-01    5.97e-01
   3  1.899774e+04    3.09e+04    1.60e+06   1.24e+02   9.77e-01  9.26e+04        1    1.24e-01    7.21e-01
   4  1.808729e+04    9.10e+02    3.97e+05   6.39e+01   9.51e-01  2.78e+05        1    1.21e-01    8.42e-01
   5  1.803399e+04    5.33e+01    1.48e+04   1.23e+01   9.99e-01  8.33e+05        1    2.02e-01    1.04e+00
   6  1.803390e+04    9.02e-02    6.35e+01   8.00e-01   1.00e+00  2.50e+06        1    1.19e-01    1.16e+00

Solver Summary (v 2.1.0-eigen-(3.4.90)-lapack-suitesparse-(5.10.1)-cxsparse-(3.2.0)-eigensparse-no_openmp-cuda-(12050))

              

In [ ]:
%cd /content

/content


In [ ]:
!mkdir MyExample
%cd MyExample

/content/MyExample


In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 2.8)
project(ch6)

set(CMAKE_BUILD_TYPE Release)
set(CMAKE_CXX_FLAGS "-std=c++14 -O3")

list(APPEND CMAKE_MODULE_PATH ${PROJECT_SOURCE_DIR}/cmake)

# OpenCV
find_package(OpenCV REQUIRED)
include_directories(${OpenCV_INCLUDE_DIRS})

# Ceres
find_package(Ceres REQUIRED)
include_directories(${CERES_INCLUDE_DIRS})

add_executable(ceresCurveFitting ceresCurveFitting.cpp)
target_link_libraries(ceresCurveFitting ${OpenCV_LIBS} ${CERES_LIBRARIES})


Writing CMakeLists.txt


In [ ]:
%%writefile ceresCurveFitting.cpp

// Created by xiang on 18-11-19.
//
#include <iostream>
#include <opencv2/core/core.hpp>
#include <ceres/ceres.h>
#include <chrono>

using namespace std;

// Calculation model of cost function
struct CURVE_FITTING_COST{
  CURVE_FITTING_COST(double x, double y) : _x(x), _y(y) {}
  // Calculation of residuals
  template<typename T>
  bool operator()(
    const T *const abc, // Model parameters, which have 3 dimensions
    T *residual) const {
    residual[0] = T(_y) - ceres::exp(abc[0] * T(_x) * T(_x) + abc[1] * T(_x) + abc[2]); // y-exp(ax^2+bx+c)
    return true;
  }

  const double _x, _y;    // x,y data
};

int main(int argc, char **argv) {
  double ar = 1.0, br = 2.0, cr = 1.0;         // True parameter value
  double ae = 2.0, be = -1.0, ce = 5.0;        // Estimated parameter values
  int N = 100;                                 // Data Points
  double w_sigma = 1.0;                        // Noise Sigma
  double inv_sigma = 1.0 / w_sigma;
  cv::RNG rng;                                 // OpenCV Random Number Generator

  vector<double> x_data, y_data;              // data
  for (int i = 0; i < N; i++) {
    double x = i / 100.0;
    x_data.push_back(x);
    y_data.push_back(exp(ar * x * x + br * x + cr) + rng.gaussian(w_sigma * w_sigma));
  }

  double abc[3] = {ae, be, ce};
  // Constructing the least squares problem
  ceres::Problem problem;
  for (int i = 0; i < N; i++) {
    problem.AddResidualBlock(

      // (cost function, loss function/Kernel function, Parameters to be estimated)

      // Adding an error term to the problem
      // Use automatic differentiation,
      // template parameters: error type, output dimension, input dimension,
      // the dimension should be consistent with the previous struct.
      new ceres::AutoDiffCostFunction<CURVE_FITTING_COST, 1, 3>
      (
        new CURVE_FITTING_COST(x_data[i], y_data[i])
      ),
      nullptr,            // Kernel function, not used here, empty
      abc                 // Parameters to be estimated
    );
  }

  // Configuring the Solver
  ceres::Solver::Options options;     // There are many configuration items to fill in here
  options.linear_solver_type = ceres::DENSE_NORMAL_CHOLESKY;  // How to solve the increment equation
  options.minimizer_progress_to_stdout = true;   // Output to cout

  ceres::Solver::Summary summary;                // Optimization information
  chrono::steady_clock::time_point t1 = chrono::steady_clock::now();
  ceres::Solve(options, &problem, &summary);  // Start Optimizing
  chrono::steady_clock::time_point t2 = chrono::steady_clock::now();
  chrono::duration<double> time_used = chrono::duration_cast<chrono::duration<double>>(t2 - t1);
  cout << "solve time cost = " << time_used.count() << " seconds. " << endl;

  // Output
  cout << summary.BriefReport() << endl;
  cout << "estimated a,b,c = ";
  for (auto a:abc) cout << a << " ";
  cout << endl;

  return 0;
}


Writing ceresCurveFitting.cpp


In [ ]:
!mkdir ceresCurveFitting_build
%cd ceresCurveFitting_build/

/content/MyExample/ceresCurveFitting_build


In [ ]:
!cmake ..

CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenCV: /usr (found version "4.5.4")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_P

In [ ]:
!make ceresCurveFitting

[ 50%] Building CXX object CMakeFiles/ceresCurveFitting.dir/ceresCurveFitting.cpp.o
[100%] Linking CXX executable ceresCurveFitting
[100%] Built target ceresCurveFitting


In [ ]:
! ./ceresCurveFitting

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.597873e+06    0.00e+00    3.52e+06   0.00e+00   0.00e+00  1.00e+04        0    3.81e-05    7.92e-05
   1  1.884440e+05    1.41e+06    4.86e+05   9.88e-01   8.82e-01  1.81e+04        1    7.20e-05    2.36e-04
   2  1.784821e+04    1.71e+05    6.78e+04   9.89e-01   9.06e-01  3.87e+04        1    4.22e-05    2.90e-04
   3  1.099631e+03    1.67e+04    8.58e+03   1.10e+00   9.41e-01  1.16e+05        1    2.72e-05    3.27e-04
   4  8.784938e+01    1.01e+03    6.53e+02   1.51e+00   9.67e-01  3.48e+05        1    2.72e-05    3.63e-04
   5  5.141230e+01    3.64e+01    2.72e+01   1.13e+00   9.90e-01  1.05e+06        1    2.69e-05    4.01e-04
   6  5.096862e+01    4.44e-01    4.27e-01   1.89e-01   9.98e-01  3.14e+06        1    2.50e-05    4.33e-04
   7  5.096851e+01    1.10e-04    9.53e-04   2.84e-03   9.99e-01  9.41e+06        1    2.69e-05    4.68e-04
solve time cost = 0.00050542

# **Chapter 8: Filters and Optimization Approaches: Part I**

**Chapter Reference:** https://github.com/gaoxiang12/slambook2/tree/master/ch9

## **Mount**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


## **Install Eigen**

In [ ]:
!git clone https://gitlab.com/libeigen/eigen.git

Cloning into 'eigen'...
remote: Enumerating objects: 125848, done.
remote: Counting objects: 100% (552/552), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 125848 (delta 360), reused 534 (delta 349), pack-reused 125296 (from 1)
Receiving objects: 100% (125848/125848), 105.61 MiB | 16.81 MiB/s, done.
Resolving deltas: 100% (104236/104236), done.


In [ ]:
%cd eigen

/content/eigen


In [ ]:
!mkdir build
%cd build

/content/eigen/build


In [ ]:
!cmake ..

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The Fortran compiler identification is GNU 11.4.0
-- Performing Test standard_math_library_linked_to_automatically
-- Performing Test standard_math_library_linked_to_automatically - Success
-- Standard libraries to link to explicitly: none
-- Performing Test COMPILER_SUPPORT_WERROR
-- Performing Test COMPILER_SUPPORT_WERROR - Success
-- Performing Test COMPILER_SUPPORT_pedantic
-- Performing Test COMPILER_SUPPORT_pedantic - Success
-- Performing Test COMPILER_SUPPORT_Wall
-- Performing T

In [ ]:
!sudo make install

[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/double.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/complex_single.cpp.o
[  0%] Building CXX object blas/CMakeFiles/eigen_blas_static.dir/xerbla.cpp.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/srotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotm.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/drotmg.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/lsame.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/dspmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/ssbmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir/f2c/chbmv.c.o
[  0%] Building C object blas/CMakeFiles/eigen_blas_static.dir

## **Install Ceres**

**TEST and Download ceres-solver-2.1.0.tar:**

https://stackoverflow.com/questions/72368717/how-to-download-ceres-solver-2-1-0-tar-gz-file-to-install-ceres-solver-in-ubun

There are **three** files of

SnavelyReprojectionError.h,

common.h,

common.cpp

that must be run before running the main part of **bundle_adjustment_ceres.cpp**

https://github.com/ceres-solver/ceres-solver

http://ceres-solver.org/installation.html

In [ ]:
%cd /content

/content


In [ ]:
!mkdir Ceres
%cd Ceres

/content/Ceres


In [ ]:
# google-glog + gflags
!sudo apt-get install libgoogle-glog-dev libgflags-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgflags2.2 libgoogle-glog0v5 libunwind-dev
The following NEW packages will be installed:
  libgflags-dev libgflags2.2 libgoogle-glog-dev libgoogle-glog0v5
  libunwind-dev
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,207 kB of archives.
After this operation, 7,675 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgflags2.2 amd64 2.2.2-2 [78.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgflags-dev amd64 2.2.2-2 [93.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgoogle-glog0v5 amd64 0.5.0+really0.4.0-2 [60.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libunwind-dev amd64 1.3.2-2build2.1 [1,883 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgoogle-glog-dev amd64 0.

In [ ]:
# Use ATLAS for BLAS & LAPACK
!sudo apt-get install libatlas-base-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
# Eigen3
!sudo apt-get install libeigen3-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  libeigen3-doc libmpfrc++-dev
The following NEW packages will be installed:
  libeigen3-dev
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,056 kB of archives.
After this operation, 9,081 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libeigen3-dev all 3.4.0-2ubuntu2 [1,056 kB]
Fetched 1,056 kB in 1s (1,061 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected p

In [ ]:
# SuiteSparse (optional)
!sudo apt-get install libsuitesparse-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2
  librbio2 libsliplu1 libspqr2 libsuitesparseconfig5 libumfpack5
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libgraphblas-dev libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2
  librbio2 libsliplu1 libspqr2 libsuitesparse-dev libsuitesparseconfig5
  libumfpack5
0 upgraded, 19 newly installed, 0 to remove and 49 not upgraded.
Need to get 22.4 MB of archives.
After this operation, 169 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5

In [ ]:
!sudo apt-get install liblapack-dev libsuitesparse-dev libcxsparse3 libgflags-dev libgoogle-glog-dev libgtest-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
libcxsparse3 is already the newest version (1:5.10.1+dfsg-4build1).
libcxsparse3 set to manually installed.
libgflags-dev is already the newest version (2.2.2-2).
libgoogle-glog-dev is already the newest version (0.5.0+really0.4.0-2).
libsuitesparse-dev is already the newest version (1:5.10.1+dfsg-4build1).
The following additional packages will be installed:
  googletest
The following NEW packages will be installed:
  googletest libgtest-dev
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 792 kB of archives.
After this operation, 5,156 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 googletest all 1.11.0-3 [541 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libgtest-dev amd64 1.11.0-3 [250 kB]
Fetched 792 kB in 0s (2,394 kB/s)
deb

In [ ]:
!git clone https://github.com/ceres-solver/ceres-solver.git

Cloning into 'ceres-solver'...
remote: Enumerating objects: 22910, done.
remote: Counting objects: 100% (912/912), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 22910 (delta 530), reused 661 (delta 435), pack-reused 21998 (from 1)
Receiving objects: 100% (22910/22910), 17.81 MiB | 16.46 MiB/s, done.
Resolving deltas: 100% (15984/15984), done.


In [ ]:
!sudo apt-get install libceres-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libceres2
The following NEW packages will be installed:
  libceres-dev libceres2
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,011 kB of archives.
After this operation, 14.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libceres2 amd64 2.0.0+dfsg1-5 [834 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libceres-dev amd64 2.0.0+dfsg1-5 [1,177 kB]
Fetched 2,011 kB in 1s (1,621 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
d

In [ ]:
# !unzip '/content/drive/MyDrive/ceres-solver-2.1.0.tar.gz' -d '/ceres-solver/Test_ceres-solver'

!tar -xzf "/content/drive/MyDrive/slambook_data/ch10/ceres-solver-2.1.0.tar.gz" -C "/content/Ceres/ceres-solver"


In [ ]:
%cd ceres-solver


# !mkdir build
# !cd build
# !checkout master
# !cmake ..
# # !cmake ../ceres-solver-2.1.0/
# !make -j32
# !sudo make install
# ORRRRR ...

/content/Ceres/ceres-solver


In [ ]:
!mkdir ceres-bin
%cd ceres-bin

/content/Ceres/ceres-solver/ceres-bin


In [ ]:
!cmake ../ceres-solver-2.1.0/

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test HAVE_BIGOBJ
-- Performing Test HAVE_BIGOBJ - Failed
-- Looking for pow in m
-- Looking for pow in m - found
-- Detected Ceres version: 2.1.0 from /content/Ceres/ceres-solver/ceres-solver-2.1.0/include/ceres/version.h
-- Detected available Ceres threading models: [CXX_THREADS, OPENMP, NO_THREADS]
-- Found Eigen version 3.4.90: /usr/local/share/eigen3/cmake
-- Enabling use of Eigen as a sparse linear algebra library.
CMake Warning (dev) at CMakeLists.txt:230 (find_package):

In [ ]:
!make -j8

[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_9.cc.o
[  0%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_4_9.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_4_3.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_d.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_4_6.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_3_6.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_4_4.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_internal.dir/generated/partitioned_matrix_view_2_4_8.cc.o
[  1%] Building CXX object internal/ceres/CMakeFiles/ceres_inter

In [ ]:
# !make help

The following are some of the valid targets for this Makefile:
... all (the default if no target is provided)
... clean
... depend
... edit_cache
... install
... install/local
... install/strip
... list_install_components
... rebuild_cache
... test
... uninstall
... array_selector_test
... array_utils_test
... autodiff_cost_function_test
... autodiff_first_order_function_test
... autodiff_local_parameterization_test
... autodiff_manifold_test
... autodiff_test
... ba_denseschur_cuda_auto_test
... ba_denseschur_cuda_auto_threads_test
... ba_denseschur_cuda_user_test
... ba_denseschur_cuda_user_threads_test
... ba_denseschur_eigen_auto_test
... ba_denseschur_eigen_auto_threads_test
... ba_denseschur_eigen_user_test
... ba_denseschur_eigen_user_threads_test
... ba_denseschur_lapack_auto_test
... ba_denseschur_lapack_auto_threads_test
... ba_denseschur_lapack_user_test
... ba_denseschur_lapack_user_threads_test
... ba_iterschur_acceleratesparse_clustjacobi_auto_test
... ba_iterschur_accele

In [ ]:
!make test

Running tests...
Test project /content/Ceres/ceres-solver/ceres-bin
        Start   1: cuda_memcheck_dense_qr_test
Could not find executable cuda-memcheck
Looked in the following places:
cuda-memcheck
cuda-memcheck
Release/cuda-memcheck
Release/cuda-memcheck
Debug/cuda-memcheck
Debug/cuda-memcheck
MinSizeRel/cuda-memcheck
MinSizeRel/cuda-memcheck
RelWithDebInfo/cuda-memcheck
RelWithDebInfo/cuda-memcheck
Deployment/cuda-memcheck
Deployment/cuda-memcheck
Development/cuda-memcheck
Development/cuda-memcheck
Unable to find executable: cuda-memcheck
  1/183 Test   #1: cuda_memcheck_dense_qr_test ...................................***Not Run   0.00 sec
        Start   2: cuda_memcheck_dense_cholesky_test
Could not find executable cuda-memcheck
Looked in the following places:
cuda-memcheck
cuda-memcheck
Release/cuda-memcheck
Release/cuda-memcheck
Debug/cuda-memcheck
Debug/cuda-memcheck
MinSizeRel/cuda-memcheck
MinSizeRel/cuda-memcheck
RelWithDebInfo/cuda-memcheck
RelWithDebInfo/cuda-memcheck
D

In [ ]:
# Optionally install Ceres, it can also be exported using CMake which
# allows Ceres to be used without requiring installation, see the documentation
# for the EXPORT_BUILD_DIR option for more information.
!make install

[ 25%] Built target ceres_internal
[ 27%] Built target ceres
[ 28%] Built target gtest
[ 28%] Built target test_util
[ 28%] Built target array_utils_test
[ 28%] Built target array_selector_test
[ 28%] Built target autodiff_test
[ 28%] Built target autodiff_first_order_function_test
[ 29%] Built target autodiff_cost_function_test
[ 29%] Built target autodiff_local_parameterization_test
[ 30%] Built target autodiff_manifold_test
[ 30%] Built target block_jacobi_preconditioner_test
[ 31%] Built target block_random_access_dense_matrix_test
[ 31%] Built target block_random_access_diagonal_matrix_test
[ 31%] Built target block_random_access_sparse_matrix_test
[ 32%] Built target block_sparse_matrix_test
[ 33%] Built target c_api_test
[ 33%] Built target canonical_views_clustering_test
[ 33%] Built target compressed_col_sparse_matrix_utils_test
[ 34%] Built target compressed_row_sparse_matrix_test
[ 34%] Built target concurrent_queue_test
[ 34%] Built target conditioned_cost_function_test
[ 3

## **Examples**

### **SLAM Book 2: BA with Ceres**

**Page 215**\
https://github.com/gaoxiang12/slambook2/blob/master/ch9/bundle_adjustment_ceres.cpp

Bundle Adjustment refers to **optimizing both** *camera parameters* (intrinsic and extrinsic) and *3D landmarks* with images. Consider the bundles of light rays emitted from 3D points. They are projected into the image planes of several cameras and then detected as feature points. The **purpose** of optimization can be explained as to **adjust** the *camera poses* and the *3D points*, to **ensure** the projected 2D features (bundles) match the detected results.

In [ ]:
%cd /content/

/content


In [ ]:
!mkdir MyBundleExample
%cd MyBundleExample

/content/MyBundleExample


In [ ]:
%%writefile random.h
#ifndef RAND_H
#define RAND_H

#include <math.h>
#include <stdlib.h>

inline double RandDouble()
{
    double r = static_cast<double>(rand());
    return r / RAND_MAX;
}

inline double RandNormal()
{
    double x1, x2, w;
    do{
        x1 = 2.0 * RandDouble() - 1.0;
        x2 = 2.0 * RandDouble() - 1.0;
        w = x1 * x1 + x2 * x2;
    }while( w >= 1.0 || w == 0.0);

    w = sqrt((-2.0 * log(w))/w);
    return x1 * w;
}

#endif // random.h

Writing random.h


In [ ]:
%%writefile rotation.h
#ifndef ROTATION_H
#define ROTATION_H

#include <algorithm>
#include <cmath>
#include <limits>

//////////////////////////////////////////////////////////////////
// math functions needed for rotation conversion.

// dot and cross production

template<typename T>
inline T DotProduct(const T x[3], const T y[3]) {
    return (x[0] * y[0] + x[1] * y[1] + x[2] * y[2]);
}

template<typename T>
inline void CrossProduct(const T x[3], const T y[3], T result[3]) {
    result[0] = x[1] * y[2] - x[2] * y[1];
    result[1] = x[2] * y[0] - x[0] * y[2];
    result[2] = x[0] * y[1] - x[1] * y[0];
}


//////////////////////////////////////////////////////////////////


// Converts from a angle anxis to quaternion :
template<typename T>
inline void AngleAxisToQuaternion(const T *angle_axis, T *quaternion) {
    const T &a0 = angle_axis[0];
    const T &a1 = angle_axis[1];
    const T &a2 = angle_axis[2];
    const T theta_squared = a0 * a0 + a1 * a1 + a2 * a2;

    if (theta_squared > T(std::numeric_limits<double>::epsilon())) {
        const T theta = sqrt(theta_squared);
        const T half_theta = theta * T(0.5);
        const T k = sin(half_theta) / theta;
        quaternion[0] = cos(half_theta);
        quaternion[1] = a0 * k;
        quaternion[2] = a1 * k;
        quaternion[3] = a2 * k;
    } else { // in case if theta_squared is zero
        const T k(0.5);
        quaternion[0] = T(1.0);
        quaternion[1] = a0 * k;
        quaternion[2] = a1 * k;
        quaternion[3] = a2 * k;
    }
}

template<typename T>
inline void QuaternionToAngleAxis(const T *quaternion, T *angle_axis) {
    const T &q1 = quaternion[1];
    const T &q2 = quaternion[2];
    const T &q3 = quaternion[3];
    const T sin_squared_theta = q1 * q1 + q2 * q2 + q3 * q3;

    // For quaternions representing non-zero rotation, the conversion
    // is numercially stable
    if (sin_squared_theta > T(std::numeric_limits<double>::epsilon())) {
        const T sin_theta = sqrt(sin_squared_theta);
        const T &cos_theta = quaternion[0];

        // If cos_theta is negative, theta is greater than pi/2, which
        // means that angle for the angle_axis vector which is 2 * theta
        // would be greater than pi...

        const T two_theta = T(2.0) * ((cos_theta < 0.0)
                                      ? atan2(-sin_theta, -cos_theta)
                                      : atan2(sin_theta, cos_theta));
        const T k = two_theta / sin_theta;

        angle_axis[0] = q1 * k;
        angle_axis[1] = q2 * k;
        angle_axis[2] = q3 * k;
    } else {
        // For zero rotation, sqrt() will produce NaN in derivative since
        // the argument is zero. By approximating with a Taylor series,
        // and truncating at one term, the value and first derivatives will be
        // computed correctly when Jets are used..
        const T k(2.0);
        angle_axis[0] = q1 * k;
        angle_axis[1] = q2 * k;
        angle_axis[2] = q3 * k;
    }

}

template<typename T>
inline void AngleAxisRotatePoint(const T angle_axis[3], const T pt[3], T result[3]) {
    const T theta2 = DotProduct(angle_axis, angle_axis);
    if (theta2 > T(std::numeric_limits<double>::epsilon())) {
        // Away from zero, use the rodriguez formula
        //
        //   result = pt costheta +
        //            (w x pt) * sintheta +
        //            w (w . pt) (1 - costheta)
        //
        // We want to be careful to only evaluate the square root if the
        // norm of the angle_axis vector is greater than zero. Otherwise
        // we get a division by zero.
        //
        const T theta = sqrt(theta2);
        const T costheta = cos(theta);
        const T sintheta = sin(theta);
        const T theta_inverse = 1.0 / theta;

        const T w[3] = {angle_axis[0] * theta_inverse,
                        angle_axis[1] * theta_inverse,
                        angle_axis[2] * theta_inverse};

        // Explicitly inlined evaluation of the cross product for
        // performance reasons.
        /*const T w_cross_pt[3] = { w[1] * pt[2] - w[2] * pt[1],
                                  w[2] * pt[0] - w[0] * pt[2],
                                  w[0] * pt[1] - w[1] * pt[0] };*/
        T w_cross_pt[3];
        CrossProduct(w, pt, w_cross_pt);

        const T tmp = DotProduct(w, pt) * (T(1.0) - costheta);
        //    (w[0] * pt[0] + w[1] * pt[1] + w[2] * pt[2]) * (T(1.0) - costheta);

        result[0] = pt[0] * costheta + w_cross_pt[0] * sintheta + w[0] * tmp;
        result[1] = pt[1] * costheta + w_cross_pt[1] * sintheta + w[1] * tmp;
        result[2] = pt[2] * costheta + w_cross_pt[2] * sintheta + w[2] * tmp;
    } else {
        // Near zero, the first order Taylor approximation of the rotation
        // matrix R corresponding to a vector w and angle w is
        //
        //   R = I + hat(w) * sin(theta)
        //
        // But sintheta ~ theta and theta * w = angle_axis, which gives us
        //
        //  R = I + hat(w)
        //
        // and actually performing multiplication with the point pt, gives us
        // R * pt = pt + w x pt.
        //
        // Switching to the Taylor expansion near zero provides meaningful
        // derivatives when evaluated using Jets.
        //
        // Explicitly inlined evaluation of the cross product for
        // performance reasons.
        /*const T w_cross_pt[3] = { angle_axis[1] * pt[2] - angle_axis[2] * pt[1],
                                  angle_axis[2] * pt[0] - angle_axis[0] * pt[2],
                                  angle_axis[0] * pt[1] - angle_axis[1] * pt[0] };*/
        T w_cross_pt[3];
        CrossProduct(angle_axis, pt, w_cross_pt);

        result[0] = pt[0] + w_cross_pt[0];
        result[1] = pt[1] + w_cross_pt[1];
        result[2] = pt[2] + w_cross_pt[2];
    }
}

#endif // rotation.h

Writing rotation.h


In [ ]:
%%writefile SnavelyReprojectionError.h

#ifndef SnavelyReprojection_H
#define SnavelyReprojection_H

#include <iostream>
#include "ceres/ceres.h"
#include "rotation.h"

class SnavelyReprojectionError {
public:
    SnavelyReprojectionError(double observation_x, double observation_y) : observed_x(observation_x),
                                                                           observed_y(observation_y) {}

    template<typename T>
    bool operator()(const T *const camera,
                    const T *const point,
                    T *residuals) const {
        // camera[0,1,2] are the angle-axis rotation
        T predictions[2];
        CamProjectionWithDistortion(camera, point, predictions);
        residuals[0] = predictions[0] - T(observed_x);
        residuals[1] = predictions[1] - T(observed_y);

        return true;
    }

    // camera : 9 dims array
    // [0-2] : angle-axis rotation
    // [3-5] : translateion
    // [6-8] : camera parameter, [6] focal length, [7-8] second and forth order radial distortion
    // point : 3D location.
    // predictions : 2D predictions with center of the image plane.
    template<typename T>
    static inline bool CamProjectionWithDistortion(const T *camera, const T *point, T *predictions) {
        // Rodrigues' formula
        T p[3];
        AngleAxisRotatePoint(camera, point, p);
        // camera[3,4,5] are the translation
        p[0] += camera[3];
        p[1] += camera[4];
        p[2] += camera[5];

        // Compute the center fo distortion
        T xp = -p[0] / p[2];
        T yp = -p[1] / p[2];

        // Apply second and fourth order radial distortion
        const T &l1 = camera[7];
        const T &l2 = camera[8];

        T r2 = xp * xp + yp * yp;
        T distortion = T(1.0) + r2 * (l1 + l2 * r2);

        const T &focal = camera[6];
        predictions[0] = focal * distortion * xp;
        predictions[1] = focal * distortion * yp;

        return true;
    }

    static ceres::CostFunction *Create(const double observed_x, const double observed_y) {
        return (new ceres::AutoDiffCostFunction<SnavelyReprojectionError, 2, 9, 3>(
            new SnavelyReprojectionError(observed_x, observed_y)));
    }

private:
    double observed_x;
    double observed_y;
};

#endif // SnavelyReprojection.h


Writing SnavelyReprojectionError.h


In [ ]:
%%writefile common.cpp

#include <cstdio>
#include <fstream>
#include <iostream>
#include <string>
#include <vector>
#include <Eigen/Core>
#include <Eigen/Dense>

#include <cassert>

#include "common.h"
#include "rotation.h"
#include "random.h"

typedef Eigen::Map<Eigen::VectorXd> VectorRef;
typedef Eigen::Map<const Eigen::VectorXd> ConstVectorRef;

template<typename T>
void FscanfOrDie(FILE *fptr, const char *format, T *value) {
    int num_scanned = fscanf(fptr, format, value);
    if (num_scanned != 1)
        std::cerr << "Invalid UW data file. ";
}

void PerturbPoint3(const double sigma, double *point) {
    for (int i = 0; i < 3; ++i)
        point[i] += RandNormal() * sigma;
}

double Median(std::vector<double> *data) {
    int n = data->size();
    std::vector<double>::iterator mid_point = data->begin() + n / 2;
    std::nth_element(data->begin(), mid_point, data->end());
    return *mid_point;
}

BALProblem::BALProblem(const std::string &filename, bool use_quaternions) {
    FILE *fptr = fopen(filename.c_str(), "r");

    if (fptr == NULL) {
        std::cerr << "Error: unable to open file " << filename;
        return;
    };

    // This wil die horribly on invalid files. Them's the breaks.
    FscanfOrDie(fptr, "%d", &num_cameras_);
    FscanfOrDie(fptr, "%d", &num_points_);
    FscanfOrDie(fptr, "%d", &num_observations_);

    std::cout << "Header: " << num_cameras_
              << " " << num_points_
              << " " << num_observations_;

    point_index_ = new int[num_observations_];
    camera_index_ = new int[num_observations_];
    observations_ = new double[2 * num_observations_];

    num_parameters_ = 9 * num_cameras_ + 3 * num_points_;
    parameters_ = new double[num_parameters_];

    for (int i = 0; i < num_observations_; ++i) {
        FscanfOrDie(fptr, "%d", camera_index_ + i);
        FscanfOrDie(fptr, "%d", point_index_ + i);
        for (int j = 0; j < 2; ++j) {
            FscanfOrDie(fptr, "%lf", observations_ + 2 * i + j);
        }
    }

    for (int i = 0; i < num_parameters_; ++i) {
        FscanfOrDie(fptr, "%lf", parameters_ + i);
    }

    fclose(fptr);

    use_quaternions_ = use_quaternions;
    if (use_quaternions) {
        // Switch the angle-axis rotations to quaternions.
        num_parameters_ = 10 * num_cameras_ + 3 * num_points_;
        double *quaternion_parameters = new double[num_parameters_];
        double *original_cursor = parameters_;
        double *quaternion_cursor = quaternion_parameters;
        for (int i = 0; i < num_cameras_; ++i) {
            AngleAxisToQuaternion(original_cursor, quaternion_cursor);
            quaternion_cursor += 4;
            original_cursor += 3;
            for (int j = 4; j < 10; ++j) {
                *quaternion_cursor++ = *original_cursor++;
            }
        }
        // Copy the rest of the points.
        for (int i = 0; i < 3 * num_points_; ++i) {
            *quaternion_cursor++ = *original_cursor++;
        }
        // Swap in the quaternion parameters.
        delete[]parameters_;
        parameters_ = quaternion_parameters;
    }
}

void BALProblem::WriteToFile(const std::string &filename) const {
    FILE *fptr = fopen(filename.c_str(), "w");

    if (fptr == NULL) {
        std::cerr << "Error: unable to open file " << filename;
        return;
    }

    fprintf(fptr, "%d %d %d %d\n", num_cameras_, num_cameras_, num_points_, num_observations_);

    for (int i = 0; i < num_observations_; ++i) {
        fprintf(fptr, "%d %d", camera_index_[i], point_index_[i]);
        for (int j = 0; j < 2; ++j) {
            fprintf(fptr, " %g", observations_[2 * i + j]);
        }
        fprintf(fptr, "\n");
    }

    for (int i = 0; i < num_cameras(); ++i) {
        double angleaxis[9];
        if (use_quaternions_) {
            //OutPut in angle-axis format.
            QuaternionToAngleAxis(parameters_ + 10 * i, angleaxis);
            memcpy(angleaxis + 3, parameters_ + 10 * i + 4, 6 * sizeof(double));
        } else {
            memcpy(angleaxis, parameters_ + 9 * i, 9 * sizeof(double));
        }
        for (int j = 0; j < 9; ++j) {
            fprintf(fptr, "%.16g\n", angleaxis[j]);
        }
    }

    const double *points = parameters_ + camera_block_size() * num_cameras_;
    for (int i = 0; i < num_points(); ++i) {
        const double *point = points + i * point_block_size();
        for (int j = 0; j < point_block_size(); ++j) {
            fprintf(fptr, "%.16g\n", point[j]);
        }
    }

    fclose(fptr);
}

// Write the problem to a PLY file for inspection in Meshlab or CloudCompare
void BALProblem::WriteToPLYFile(const std::string &filename) const {
    std::ofstream of(filename.c_str());

    of << "ply"
       << '\n' << "format ascii 1.0"
       << '\n' << "element vertex " << num_cameras_ + num_points_
       << '\n' << "property float x"
       << '\n' << "property float y"
       << '\n' << "property float z"
       << '\n' << "property uchar red"
       << '\n' << "property uchar green"
       << '\n' << "property uchar blue"
       << '\n' << "end_header" << std::endl;

    // Export extrinsic data (i.e. camera centers) as green points.
    double angle_axis[3];
    double center[3];
    for (int i = 0; i < num_cameras(); ++i) {
        const double *camera = cameras() + camera_block_size() * i;
        CameraToAngelAxisAndCenter(camera, angle_axis, center);
        of << center[0] << ' ' << center[1] << ' ' << center[2]
           << " 0 255 0" << '\n';
    }

    // Export the structure (i.e. 3D Points) as white points.
    const double *points = parameters_ + camera_block_size() * num_cameras_;
    for (int i = 0; i < num_points(); ++i) {
        const double *point = points + i * point_block_size();
        for (int j = 0; j < point_block_size(); ++j) {
            of << point[j] << ' ';
        }
        of << " 255 255 255\n";
    }
    of.close();
}

void BALProblem::CameraToAngelAxisAndCenter(const double *camera,
                                            double *angle_axis,
                                            double *center) const {
    VectorRef angle_axis_ref(angle_axis, 3);
    if (use_quaternions_) {
        QuaternionToAngleAxis(camera, angle_axis);
    } else {
        angle_axis_ref = ConstVectorRef(camera, 3);
    }

    // c = -R't
    Eigen::VectorXd inverse_rotation = -angle_axis_ref;
    AngleAxisRotatePoint(inverse_rotation.data(),
                         camera + camera_block_size() - 6,
                         center);
    VectorRef(center, 3) *= -1.0;
}

void BALProblem::AngleAxisAndCenterToCamera(const double *angle_axis,
                                            const double *center,
                                            double *camera) const {
    ConstVectorRef angle_axis_ref(angle_axis, 3);
    if (use_quaternions_) {
        AngleAxisToQuaternion(angle_axis, camera);
    } else {
        VectorRef(camera, 3) = angle_axis_ref;
    }

    // t = -R * c
    AngleAxisRotatePoint(angle_axis, center, camera + camera_block_size() - 6);
    VectorRef(camera + camera_block_size() - 6, 3) *= -1.0;
}

void BALProblem::Normalize() {
    // Compute the marginal median of the geometry
    std::vector<double> tmp(num_points_);
    Eigen::Vector3d median;
    double *points = mutable_points();
    for (int i = 0; i < 3; ++i) {
        for (int j = 0; j < num_points_; ++j) {
            tmp[j] = points[3 * j + i];
        }
        median(i) = Median(&tmp);
    }

    for (int i = 0; i < num_points_; ++i) {
        VectorRef point(points + 3 * i, 3);
        tmp[i] = (point - median).lpNorm<1>();
    }

    const double median_absolute_deviation = Median(&tmp);

    // Scale so that the median absolute deviation of the resulting
    // reconstruction is 100

    const double scale = 100.0 / median_absolute_deviation;

    // X = scale * (X - median)
    for (int i = 0; i < num_points_; ++i) {
        VectorRef point(points + 3 * i, 3);
        point = scale * (point - median);
    }

    double *cameras = mutable_cameras();
    double angle_axis[3];
    double center[3];
    for (int i = 0; i < num_cameras_; ++i) {
        double *camera = cameras + camera_block_size() * i;
        CameraToAngelAxisAndCenter(camera, angle_axis, center);
        // center = scale * (center - median)
        VectorRef(center, 3) = scale * (VectorRef(center, 3) - median);
        AngleAxisAndCenterToCamera(angle_axis, center, camera);
    }
}

void BALProblem::Perturb(const double rotation_sigma,
                         const double translation_sigma,
                         const double point_sigma) {
    assert(point_sigma >= 0.0);
    assert(rotation_sigma >= 0.0);
    assert(translation_sigma >= 0.0);

    double *points = mutable_points();
    if (point_sigma > 0) {
        for (int i = 0; i < num_points_; ++i) {
            PerturbPoint3(point_sigma, points + 3 * i);
        }
    }

    for (int i = 0; i < num_cameras_; ++i) {
        double *camera = mutable_cameras() + camera_block_size() * i;

        double angle_axis[3];
        double center[3];
        // Perturb in the rotation of the camera in the angle-axis
        // representation
        CameraToAngelAxisAndCenter(camera, angle_axis, center);
        if (rotation_sigma > 0.0) {
            PerturbPoint3(rotation_sigma, angle_axis);
        }
        AngleAxisAndCenterToCamera(angle_axis, center, camera);

        if (translation_sigma > 0.0)
            PerturbPoint3(translation_sigma, camera + camera_block_size() - 6);
    }
}

Writing common.cpp


We use the **BAL dataset** to demonstrate the BA experiments. The BAL data set
provides several scenes. The camera and landmark information in each scene are
given by a text file.

We use the file problem-16-22106-pre.txt as an example. This file
stores the BA problem information in a line-by-line manner. For the detailed format,
see https://grail.cs.washington.edu/projects/bal. We use the BALProblem
class defined in common.h to read in the content of the file, and then use Ceres and
g2o to solve them.

After reading the data with the BALProblem class, we can call the Normalize function to normalize the original data, or add noise to the data through the Perturb function. Normalization means setting zero to the centers of all landmarks, and then scaling them to an appropriate scale. This will make the value in the optimization process more stable and prevent BA to get very large values in extreme cases.

In [ ]:
%%writefile common.h

#pragma once

/// 从文件读入BAL dataset
class BALProblem {
public:
    /// load bal data from text file
    explicit BALProblem(const std::string &filename, bool use_quaternions = false);

    ~BALProblem() {
        delete[] point_index_;
        delete[] camera_index_;
        delete[] observations_;
        delete[] parameters_;
    }

    /// save results to text file
    void WriteToFile(const std::string &filename) const;

    /// save results to ply pointcloud
    void WriteToPLYFile(const std::string &filename) const;

    void Normalize();

    void Perturb(const double rotation_sigma,
                 const double translation_sigma,
                 const double point_sigma);

    int camera_block_size() const { return use_quaternions_ ? 10 : 9; }

    int point_block_size() const { return 3; }

    int num_cameras() const { return num_cameras_; }

    int num_points() const { return num_points_; }

    int num_observations() const { return num_observations_; }

    int num_parameters() const { return num_parameters_; }

    const int *point_index() const { return point_index_; }

    const int *camera_index() const { return camera_index_; }

    const double *observations() const { return observations_; }

    const double *parameters() const { return parameters_; }

    const double *cameras() const { return parameters_; }

    const double *points() const { return parameters_ + camera_block_size() * num_cameras_; }

    /// camera参数的起始地址
    double *mutable_cameras() { return parameters_; }

    double *mutable_points() { return parameters_ + camera_block_size() * num_cameras_; }

    double *mutable_camera_for_observation(int i) {
        return mutable_cameras() + camera_index_[i] * camera_block_size();
    }

    double *mutable_point_for_observation(int i) {
        return mutable_points() + point_index_[i] * point_block_size();
    }

    const double *camera_for_observation(int i) const {
        return cameras() + camera_index_[i] * camera_block_size();
    }

    const double *point_for_observation(int i) const {
        return points() + point_index_[i] * point_block_size();
    }

private:
    void CameraToAngelAxisAndCenter(const double *camera,
                                    double *angle_axis,
                                    double *center) const;

    void AngleAxisAndCenterToCamera(const double *angle_axis,
                                    const double *center,
                                    double *camera) const;

    int num_cameras_;
    int num_points_;
    int num_observations_;
    int num_parameters_;
    bool use_quaternions_;

    int *point_index_;      // 每个observation对应的point index
    int *camera_index_;     // 每个observation对应的camera index
    double *observations_;
    double *parameters_;
};

Writing common.h


In [ ]:
%%writefile CMakeLists.txt

cmake_minimum_required(VERSION 2.8)

project(bundle_adjustment)
set(CMAKE_BUILD_TYPE "Release")
set(CMAKE_CXX_FLAGS "-O3 -std=c++14")

LIST(APPEND CMAKE_MODULE_PATH ${PROJECT_SOURCE_DIR}/cmake)

include_directories("/content/eigen")
include_directories("/content/Ceres")
#include_directories("/content/ceres-bin")


Find_Package(Eigen3 REQUIRED)
Find_Package(Ceres REQUIRED)

include_directories(${PROJECT_SOURCE_DIR} ${EIGEN3_INCLUDE_DIR} ${CSPARSE_INCLUDE_DIR})

add_library(bal_common common.cpp)
add_executable(bundle_adjustment_ceres bundle_adjustment_ceres.cpp)

target_link_libraries(bundle_adjustment_ceres ${CERES_LIBRARIES} bal_common)


Writing CMakeLists.txt


In [ ]:
%%writefile bundle_adjustment_ceres.cpp

#include <iostream>
#include <ceres/ceres.h>
#include "common.h"
// #include </content/eigen/blas/common.h>
// #include "SnavelyReprojectionError.h"
#include "/content/MyBundleExample/SnavelyReprojectionError.h"
// #include </content/Ceres/ceres-solver/examples/snavely_reprojection_error.h>

using namespace std;

void SolveBA(BALProblem &bal_problem);

// BA problem main part
int main(int argc, char **argv){
    if (argc != 2) {
        cout << "usage: bundle_adjustment_ceres bal_data.txt" << endl;
        return 1;
    }

    BALProblem bal_problem(argv[1]);
    bal_problem.Normalize();
    bal_problem.Perturb(0.1, 0.5, 0.5);
    bal_problem.WriteToPLYFile("initial.ply");
    SolveBA(bal_problem);
    bal_problem.WriteToPLYFile("final.ply");

    return 0;
}

// Set BA problem
void SolveBA(BALProblem &bal_problem) {
    const int point_block_size = bal_problem.point_block_size();
    const int camera_block_size = bal_problem.camera_block_size();
    double *points = bal_problem.mutable_points();
    double *cameras = bal_problem.mutable_cameras();

    // Observations is 2 * num_observations long array observations
    // [u_1, u_2, ... u_n], where each u_i is two dimensional,
    // the x and y position of the observation.
    const double *observations = bal_problem.observations();
    ceres::Problem problem;

    for (int i = 0; i < bal_problem.num_observations(); ++i) {
        ceres::CostFunction *cost_function;

        // Each Residual block takes a point and a camera as input
        // and outputs a 2 dimensional Residual
        cost_function = SnavelyReprojectionError::Create(observations[2 * i + 0], observations[2 * i + 1]);

        // If enabled use Huber's loss function.
        ceres::LossFunction *loss_function = new ceres::HuberLoss(1.0);

        // Each observation corresponds to a pair of a camera and a point
        // which are identified by camera_index()[i] and point_index()[i]
        // respectively.
        double *camera = cameras + camera_block_size * bal_problem.camera_index()[i];
        double *point = points + point_block_size * bal_problem.point_index()[i];
        // (cost function, loss function/Kernel function, Parameters to be estimated)
        problem.AddResidualBlock(cost_function, loss_function, camera, point);
    }

    // show some information here ...
    std::cout << "bal problem file loaded..." << std::endl;
    std::cout << "bal problem have " << bal_problem.num_cameras() << " cameras and "
              << bal_problem.num_points() << " points. " << std::endl;
    std::cout << "Forming " << bal_problem.num_observations() << " observations. " << std::endl;

    std::cout << "Solving ceres BA ... " << endl;
    ceres::Solver::Options options;
    options.linear_solver_type = ceres::LinearSolverType::SPARSE_SCHUR;
    options.minimizer_progress_to_stdout = true;
    ceres::Solver::Summary summary;
    ceres::Solve(options, &problem, &summary);
    std::cout << summary.FullReport() << "\n";
}

Writing bundle_adjustment_ceres.cpp


In [ ]:
!mkdir build
%cd build/

/content/MyBundleExample/build


In [ ]:
!cmake ..

CMake Deprecation Warning at CMakeLists.txt:2 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Fou

In [ ]:
!make bundle_adjustment_ceres

[ 25%] Building CXX object CMakeFiles/bal_common.dir/common.cpp.o
[ 50%] Linking CXX static library libbal_common.a
[ 50%] Built target bal_common
[ 75%] Building CXX object CMakeFiles/bundle_adjustment_ceres.dir/bundle_adjustment_ceres.cpp.o
[100%] Linking CXX executable bundle_adjustment_ceres
[100%] Built target bundle_adjustment_ceres


In [ ]:
! ./bundle_adjustment_ceres

usage: bundle_adjustment_ceres bal_data.txt


We use the file **problem-16-22106-pre.txt** as an example. This file
stores the BA problem information in a line-by-line manner. For the detailed format,
see https://grail.cs.washington.edu/projects/bal. We use the BALProblem
class defined in **common.h** to read in the content of the file, and then use Ceres and
g2o to solve them.

It should be noted that the BAL data set has some special features:
* The camera intrinsic model of BAL is given by the focal length f and the
distortion parameters k1, k2, where f is same as the fx and fy mentioned
before. Since the pixels of the photo are basically square, in many practical
situations, fx is very close to fy, so it is possible to use the same value. In
addition, there is no cx, cy in this model, because these two values have been
removed from the stored data.


* BAL data assumes that the projection plane is behind the optical center of
the camera when projecting, so if we calculate according to the model we used
before, we need to multiply -1 after projection. However, most data sets still
use the projection plane in front of the optical center. We should read the
format description carefully before using the data set.

In [ ]:
# Example
# !bin/simple_bundle_adjuster ../ceres-solver-2.1.0/data/problem-16-22106-pre.txt

!./bundle_adjustment_ceres /content/Ceres/ceres-solver/ceres-solver-2.1.0/data/problem-16-22106-pre.txt

Header: 16 22106 83718bal problem file loaded...
bal problem have 16 cameras and 22106 points. 
Forming 83718 observations. 
Solving ceres BA ... 
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  1.842900e+07    0.00e+00    2.04e+06   0.00e+00   0.00e+00  1.00e+04        0    6.43e-02    2.11e-01
   1  1.449093e+06    1.70e+07    1.75e+06   2.16e+03   1.84e+00  3.00e+04        1    3.26e-01    5.37e-01
   2  5.848543e+04    1.39e+06    1.30e+06   1.55e+03   1.87e+00  9.00e+04        1    2.80e-01    8.17e-01
   3  1.581483e+04    4.27e+04    4.98e+05   4.98e+02   1.29e+00  2.70e+05        1    1.87e-01    1.01e+00
   4  1.251823e+04    3.30e+03    4.64e+04   9.96e+01   1.11e+00  8.10e+05        1    3.21e-01    1.33e+00
   5  1.240936e+04    1.09e+02    9.78e+03   1.33e+01   1.42e+00  2.43e+06        1    2.56e-01    1.59e+00
   6  1.237699e+04    3.24e+01    3.91e+03   5.04e+00   1.70e+00  7.29e+06        1    2.57e-01  

# **More Examples and Useful Links**

http://ceres-solver.org/tutorial.html

https://github.com/ceres-solver/ceres-solver

https://github.com/ceres-solver/ceres-solver/blob/master/examples

http://ceres-solver.org/tutorial.html

http://ceres-solver.org/nnls_tutorial.html